In [1]:
from transformers import pipeline
import pandas as pd
import numpy as np

2022-12-08 18:25:20.006161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 18:25:23.688687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 18:25:23.688730: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-08 18:25:31.730507: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
!nvidia-smi

Thu Dec  8 18:31:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   31C    P0    27W / 250W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
df = pd.read_csv('lyrics_data.csv')
df.head()

,Unnamed: 0,genre,lyrics
0,0,pop,hold time feel break feel untrue convince spea...
1,1,pop,believe drop rain fall grow believe darkest ni...
2,2,pop,sweetheart send letter goodbye secret feel bet...
3,3,pop,kiss lips want stroll charm mambo chacha merin...
4,4,pop,till darling till matter know till dream live ...


In [17]:
df['lyrics'] = df.lyrics.str.replace('[^ \w+]', '')

/tmp/ipykernel_7482/678373657.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['lyrics'] = df.lyrics.str.replace('[^ \w+]', '')


In [18]:
df

,Unnamed: 0,genre,lyrics
0,0,pop,hold time feel break feel untrue convince spea...
1,1,pop,believe drop rain fall grow believe darkest ni...
2,2,pop,sweetheart send letter goodbye secret feel bet...
3,3,pop,kiss lips want stroll charm mambo chacha merin...
4,4,pop,till darling till matter know till dream live ...
...,...,...,...
28367,28367,hip hop,cause fuck leave scar tick tock clock come kno...
28368,28368,hip hop,minks things chain ring braclets yap fame come...
28369,28369,hip hop,get ban get ban stick crack relax plan attack ...
28370,28370,hip hop,check check yeah yeah hear thing call switch g...


In [4]:
df['genre'].value_counts()

pop        7042
country    5445
blues      4604
rock       4034
jazz       3845
reggae     2498
hip hop     904
Name: genre, dtype: int64

In [4]:
possible_labels = df.genre.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'pop': 0,
 'country': 1,
 'blues': 2,
 'jazz': 3,
 'reggae': 4,
 'rock': 5,
 'hip hop': 6}

In [19]:
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')
def roberta(x):
    sequence_to_classify = x
    candidate_labels = possible_labels
    a = classifier(sequence_to_classify, candidate_labels)
    return candidate_labels[np.argmax(a['scores'])]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
df['roberta'] = df['lyrics'].apply(lambda x: roberta(x))

Bad pipe message: %s [b"*\xa9\x03\x96\x135\xe9%\x10\xe3\xcb\x8f\xc7\x1f\xf4\xa2\x8eE\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08"]
Bad pipe message: %s [b'\n\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\x02\x03', b'\x02\x01', b'\x02\x02']
Bad pipe message: %s [b"\xe1Jdj\xf4\x1d\xc6m\xd9,\x11\x82\x17\x94Wl\xdfw\x00\x00\xa